# Building a song recommender

Context : Dear xxxxxxxx,

We are thrilled to welcome you as a Data Analyst for Gnoosic!

As you know, we are trying to come up with ways to enhance our music recommendations. One of the new features we'd like to research is to recommend songs (not only bands). We're also aware of the limitations of our collaborative filtering algorithms, and would like to give users two new possibilities when searching for recommendations:

Songs that are actually similar to the ones they picked from an acoustic point of view.
Songs that are popular around the world right now, independently from their tastes.
Coming up with the perfect song recommender will take us months - no need to stress out too much. In this first week, we want you to explore new data sources for songs. The internet is full of information and our first step is to acquire it do an initial exploration. Feel free to use APIs or directly scrape the web to collect as much information as possible from popular songs. Eventually, we'll need to collect data from millions of songs, but we can start with a few hundreds or thousands from each source and see if the collected features are useful. 

Once the data is collected, we want you to create clusters of songs that are similar to each other. The idea is that if a user inputs a song from one group, we'll prioritize giving them recommendations of songs from that same group.

On Friday, you will present your work to me and Marek, the CEO and founder. Full disclosure: I need you to be very convincing about this whole song-recommender, as this has been my personal push and the main reason we hired you for!

Be open minded about this process: we are agile, and that means that we define our products and features on-the-go, while exploring the tools and the data that's available to us. We'd love you to provide your own vision of the product and the next steps to be taken.

Lots of luck and strength for this first week with us!

Jane

In [48]:
def Gnoosic_recommender ():
    
    #importing libraries
    import pandas as pd
    from pandas import json_normalize
    import spotipy 
    from spotipy.oauth2 import SpotifyClientCredentials
    import pickle
    from IPython.display import IFrame
    
    #importing databases
    top100_df = pd.read_csv('hot_songs.csv')
    top_10000= pd.read_csv('clustered_songs.csv')
    
    #connecting to spotify
    secrets_file = open(".secret.txt","r")
    string = secrets_file.read()
    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))
    #beginning of recommendation process
    print("Hi! Looking for song recommendations ? Leave it to me :D")
    
    #identify user input
    song = input("Enter a song title: ")
    song=song.strip().lower()
    artist = input("Enter the name of the artist: ")
    artist=artist.strip().lower()
    user_result = sp.search(q="artist:" + artist + " track:" + song, type="track", limit=1)
    tracks = json_normalize(user_result["tracks"]["items"])
    
    #identify wrong input
    if user_result['tracks']['items'] == []:
        print ('Oops, try another song or check for typos')
        
    #recommend hot songs
    elif song in top100_df['title'].str.lower().values and artist in top100_df['artist'].str.lower().values:
            #select a random song
        random=top100_df.sample()
            #searching the song in spotify to produce a clikable link
        random_result = sp.search (q="artist:" + random['artist'].values[0] + 
                                   " track:" + random['title'].values[0], type="track", limit=1)
        random_song = json_normalize(random_result["tracks"]["items"])
        song_id = random_song['id'].values[0]
        print("You should check out this hot song:")
        print(random['title'].values[0], ' by ', random['artist'].values[0])
        display(IFrame(src=f"https://open.spotify.com/embed/track/{song_id}",
                       width="320",
                       height="80",
                       frameborder="0",
                       allowtransparency="true",
                       allow="encrypted-media",
                      ))
        
    else:
        #recommend top 10000 songs
            #extracting audio features
        audio_features_list = sp.audio_features(tracks['id'][0])
        audio_features_df = json_normalize(audio_features_list)
        user_df = pd.concat([tracks["popularity"], audio_features_df], axis=1)
        num_df=user_df.select_dtypes('number')
            #using saved scaler and cluster model
        scale = pickle.load(open('scaler_model.pkl','rb'))
        num_scaled = scale.transform(num_df)
        data_scaled = pd.DataFrame(num_scaled, columns=num_df.columns)
        model = pickle.load(open('clustering_model.pkl','rb'))
        user_cluster = model.predict(data_scaled)
            #retrieving cluster number
        user_cluster = user_cluster[0]
            #selecting songs subset from the same cluster
        songs_cluster= top_10000[top_10000["cluster"]==user_cluster]
            #recommending random song from the cluster
        random=songs_cluster.sample()
        song_id=random['song_id'].values[0]
        print("You should check out:")
        print(random['song_name'].values[0], ' by ', random['name'].values[0])
            #generate clikable link
        display(IFrame(src=f"https://open.spotify.com/embed/track/{song_id}",
                       width="320",
                       height="80",
                       frameborder="0",
                       allowtransparency="true",
                       allow="encrypted-media",
                      ))
        return
    

In [49]:
Gnoosic_recommender()

Hi! Looking for song recommendations ? Leave it to me :D
Enter a song title:     NOT LIKE US
Enter the name of the artist: kendRICK LAMAR
You should check out this hot song:
I Can't  by  Reba McEntire


In [50]:
Gnoosic_recommender()

Hi! Looking for song recommendations ? Leave it to me :D
Enter a song title: i wANT it that WAY
Enter the name of the artist: Backstreet Boys
You should check out:
The Days Of Pearly Spencer  by  Marc Almond


In [51]:
Gnoosic_recommender()

Hi! Looking for song recommendations ? Leave it to me :D
Enter a song title: SOOOwoojah
Enter the name of the artist: ABABABABABAB
Oops, try another song or check for typos


In [52]:
Gnoosic_recommender()

Hi! Looking for song recommendations ? Leave it to me :D
Enter a song title: Gangam style
Enter the name of the artist: Psy
Oops, try another song or check for typos


In [53]:
Gnoosic_recommender()

Hi! Looking for song recommendations ? Leave it to me :D
Enter a song title: Gangnam style
Enter the name of the artist: Psy
You should check out:
Move on Up - Single Edit  by  Curtis Mayfield
